# Word Embedding
### Gender bias: Science VS Art

In [1]:
%pip install numpy pandas gensim matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 5.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 21.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 9.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 10.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 3.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/1

In [2]:
# Core packages for word embeddings
import gensim
from gensim.models import Word2Vec
import gensim.downloader as api


In [4]:
# more pachages
import numpy as np  # For creating the embedding matrix
# prepare a small model
model_name = 'glove-twitter-25'
w2v_small = api.load(model_name)

[==================================================] 100.0% 104.8/104.8MB downloaded


In [6]:
# Define the target and attribute word sets
target_male = ['he', 'his', 'him', 'son', 'father', 'boy', 'brother', 'man', 'male']
target_female = ['she', 'her', 'hers', 'daughter', 'mother', 'girl', 'sister', 'woman', 'female']
attribute_science = ['physics', 'chemistry', 'biology', 'mathematics', 'research',
                  'experiment', 'scientist', 'lab', 'hypothesis', 'discovery']
attribute_arts = ['music', 'painting', 'literature', 'poetry', 'dance',
               'sculpture', 'theater', 'drawing', 'creativity', 'performance']

In [7]:
# Calculate the WEAT score
def cosine_similarity(word, attribute_set):
        word_vector = w2v_small[word]
        attribute_vectors = [w2v_small[attr] for attr in attribute_set]
        return np.mean([np.dot(word_vector, attr_vector) / (np.linalg.norm(word_vector) * np.linalg.norm(attr_vector))
                        for attr_vector in attribute_vectors])

def weat_score(word_set_X, word_set_Y, word_set_A, word_set_B):

    X_association = np.mean([cosine_similarity(x, attribute_science) - cosine_similarity(x, attribute_arts)
                             for x in word_set_X])
    Y_association = np.mean([cosine_similarity(y, attribute_science) - cosine_similarity(y, attribute_arts)
                             for y in word_set_Y])
    A_association = np.mean([cosine_similarity(a, attribute_science) - cosine_similarity(a, attribute_arts)
                             for a in word_set_A])
    B_association = np.mean([cosine_similarity(b, attribute_science) - cosine_similarity(b, attribute_arts)
                             for b in word_set_B])

    return X_association - Y_association / np.sqrt(np.var([X_association, Y_association]) +
                                                    np.var([A_association, B_association]))


In [8]:
# Calculate the WEAT score for the given word sets
weat_result = weat_score(target_male, target_female, attribute_science, attribute_arts)

# Step 4: Print the WEAT result
print(f"WEAT score: {weat_result}")

WEAT score: 0.8146985173225403


In [10]:
?np.var

Signature:
np.var(
    a,
    axis=None,
    dtype=None,
    out=None,
    ddof=0,
    keepdims=<no value>,
    *,
    where=<no value>,
)
Docstring:
Compute the variance along the specified axis.

Returns the variance of the array elements, a measure of the spread of a
distribution.  The variance is computed for the flattened array by
default, otherwise over the specified axis.

Parameters
----------
a : array_like
    Array containing numbers whose variance is desired.  If `a` is not an
    array, a conversion is attempted.
axis : None or int or tuple of ints, optional
    Axis or axes along which the variance is computed.  The default is to
    compute the variance of the flattened array.

    .. versionadded:: 1.7.0

    If this is a tuple of ints, a variance is performed over multiple axes,
    instead of a single axis or all the axes as before.
dtype : data-type, optional
    Type to use in computing the variance.  For arrays of integer type
    the default is `float64`; for arra